In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 14.2 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from transformers import BitsAndBytesConfig,GemmaTokenizer

In [ ]:
hf_token = userdata.get('hf_token')
os.environ['HF_TOKEN'] = hf_token
login(token=hf_token)
os.environ['WANDB_DISABLED'] = 'True'

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
model_name='google/gemma-2b'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map={"":0}
                                             )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules=['q_proj','o_proj','k_proj','v_proj',
                    'gate_proj','up_proj','down_proj'],
    task_type ="CAUSAL_LM"
)

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
data = pd.read_csv('/content/data .csv')
data = Dataset.from_pandas(data)
data =data.map(lambda samples:tokenizer([str(x) for x in samples['process']]),batched=True)

Map:   0%|          | 0/89 [00:00<?, ? examples/s]

In [ ]:
data['process']

Column(['Sim change එකක් කරගනිද්දි original customer brunch එකට visit කරන්න ඔනේ. අනිවර්යෙන්ම original customer NIC හො Driving license එක අරගෙන brunch එකට යන්න ඕනේ. අපි හිතමු NIC එක කියලා unclear හරි dammage අපිට NIC එක verified කරන්න බෑ .ඒ විතරක් නෙමෙයි NIC එක නැත්නම් අපි Driving license  එක ගෙනියන්ව කියමුකො එකගෙනිච්චත්  same එත් එක clear තියෙන්න ඔනේ. එක expired වෙලා තියෙන්න බෑ incase expire වෙලානම් අපි එක accept  කරන්නෙත් නැ. ඔය දෙකම ලග නැත්නම් අපිට එයාට option එකක් දෙන්න පුලුවන් ඒ තමය් Passport එක. ඒත් එකත් expired වෙලා තියෙන්න බෑ. තව perority customer කෙනෙක් නෙමෙයි නම් රුපියල් 100ක් change වෙනවා ඒ වගේම තමන්ගි නමට SIM එක තියෙන්නත් ඔනේ සර් . Sim එක active වෙන්න පැය 3ක් වගේ යනවා.', 'Original customer branch එකට visit කරන්න ඔනේ එයගේ NIC, Driving license or Passport  එකත් එක්ක ඒ වගේම රුපියක් 100 charge වෙනවා. පැය 3 වගේ කාලයක් යනවා. ඊට පස්සේ තමයි ඔබතුමාගෙ SIM  එක phone  එකට දාගන්න ඕනේ.', 'Sim change එක කරන්න original customer brunch එකට එන්න ඔනේ brunch එකට එනකොට NIC එක හරි Driving license

In [ ]:
def formatting_func(example):
  text=f"quate: {example['quote'][0]}\nAuthor:{example['author'][0]}"
  return[text]

In [ ]:
data['Trasaction']

Column(['sim change එකක් කරන්නෙ කෙසේද?', 'sim change එකක් කරන්නෙ කෙසේද?', 'sim change එකක් කරන්නෙ කෙසේද?', 'sim change එකක් කරන්නෙ කෙසේද?', 'sim change එකක් කරන්නෙ කෙසේද?'])

In [ ]:
trainer =SFTTrainer(model=model,
                    train_dataset=data,
                    args=transformers.TrainingArguments(
                        per_device_train_batch_size=1,
                        gradient_accumulation_steps=4,
                        warmup_steps=2,
                        max_steps=100,
                        learning_rate=2e-4,
                        bf16=True,
                        logging_steps=1,
                        output_dir='outputs',
                        optim='paged_adamw_8bit',
                        report_to='none'


                    ),
                    peft_config=lora_config,
                    formatting_func=formatting_func,)

Truncating train dataset:   0%|          | 0/89 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
1,2.625400
2,2.596800
3,2.815400
4,2.602100
5,2.269700
6,2.633100
7,2.389400
8,2.142800
9,2.030300
10,2.013900


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=100, training_loss=1.2027819338440895, metrics={'train_runtime': 500.249, 'train_samples_per_second': 0.8, 'train_steps_per_second': 0.2, 'total_flos': 770174665666560.0, 'train_loss': 1.2027819338440895, 'epoch': 4.359550561797753})

In [ ]:
text = 'Quate: Sim change එකක් කරන්නෙ කෙසේද?'
device = "cuda:0"
inputs =tokenizer(text,return_tensors='pt').to(device)

outputs =model.generate(**inputs,max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quate: Sim change එකක් කරන්නෙ කෙසේද? එයට branch එකට පැමිණෙන විට, branch එකට පැමිණෙන customer තමන්ගේ valid NIC හෝ Driving License එකක් තිබිය යුතුය. Passport එකක් භාවිතා කරන විට, අලුත්ම මාස 3ක් ඇතුළත billing proof එකක් (Light bill, Water bill වැනි) ඉදිරිපත් කළ යුතුය. Reload කරන SIM එකක් ගෙන එමත් අනිවාර්යයි. Priority customer නොවන පුද්ගලයන්ට Rs.100 ක් reload fee ගෙවිය යුතුය. Priority customer නම


In [ ]:
full_model = trainer.model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
full_model.save_pretrained("./gemma-2-9b-finetuned")
tokenizer.save_pretrained("./gemma-2-9b-finetuned")

('./gemma-2-9b-finetuned/tokenizer_config.json',
 './gemma-2-9b-finetuned/special_tokens_map.json',
 './gemma-2-9b-finetuned/tokenizer.model',
 './gemma-2-9b-finetuned/added_tokens.json',
 './gemma-2-9b-finetuned/tokenizer.json')

In [ ]:
print(os.listdir("./gemma-2-9b-finetuned"))

['special_tokens_map.json', 'tokenizer_config.json', 'config.json', 'model.safetensors', 'tokenizer.model', 'tokenizer.json', 'generation_config.json']
